In [1]:
import pandas as pd
import numpy as np

df = pd.read_table('~/Downloads/INFO_4604/all_tweets_withprobabilities.csv', sep=',', header=None)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,274586346239164416,@avalenc this was a tetanus vaccine too! I tol...,0.8597,negative,2012-11-30,South San Francisco,San Mateo County,California,United States,F,0.555259
1,1,273088776564244482,@nancyholtzman flu shots for 6MO + mom? and ho...,0.0508,negative,2012-11-26,Cincinnati,Hamilton County,Ohio,United States,F,0.025630
2,2,279187527167000576,I'm wondering why we all got flu shots since m...,0.5721,negative,2012-12-13,x,x,x,x,M,0.623470
3,3,279187825801441280,Use the Flu Vaccine Finder to find a flu vacci...,0.0014,negative,2012-12-13,Asheville,Buncombe County,North Carolina,United States,M,0.008960
4,4,279188167368781825,"Cool, today's only Thursday and I have to get ...",0.9978,negative,2012-12-13,x,x,x,x,M,0.966922


In [2]:
df2 = df.loc[:,[1,2,5]]

df3 = df2.rename(index=str, columns={1: "ID", 2: "Text", 5: "Date"})

df4 = pd.read_table('~/Downloads/INFO_4604/training_tweets_meta.tsv', sep='\t', header=0)

df5 = pd.merge(df4,df3,left_on='TweetID',right_on='ID',how='outer')

df6 = df5.loc[:,['ID','Text','Intent','Date_x']]
df7 = df6.rename(index=str, columns={"Intent": "Illness",'Date_x':'Date'})

df7.head()

,ID,Text,Illness,Date
0,5.226010e+17,MedSpring's new River North clinic to offer fr...,no,2014-10-16
1,5.226452e+17,Jk. Flu shots be saving lives but they're anno...,no,2014-10-16
2,5.226622e+17,"i will get the flu shot anyway, not cause ebol...",yes,2014-10-16
3,5.226739e+17,"Speaking of flu shots, I found out I work with...",no,2014-10-16
4,5.227409e+17,Who smiles for a flu shot? WWj producer Terri ...,no,2014-10-16


In [3]:
df7.replace('no', '0', inplace = True)
df7.replace('yes', '1', inplace = True)
        
df7.head()

,ID,Text,Illness,Date
0,5.226010e+17,MedSpring's new River North clinic to offer fr...,0,2014-10-16
1,5.226452e+17,Jk. Flu shots be saving lives but they're anno...,0,2014-10-16
2,5.226622e+17,"i will get the flu shot anyway, not cause ebol...",1,2014-10-16
3,5.226739e+17,"Speaking of flu shots, I found out I work with...",0,2014-10-16
4,5.227409e+17,Who smiles for a flu shot? WWj producer Terri ...,0,2014-10-16


In [4]:
df7 = df7.dropna()

In [5]:
from collections import Counter
import re

month = re.compile('[2][\d][\d][\d][-]([\d]+)(?:[-])')

months = Counter()

for index, row in df7.iterrows():
    date = row['Date']
    m = month.search(date)
    for group in m.groups():
        months[group] += 1
    
months

Counter({'1': 1973,
         '10': 2304,
         '11': 1203,
         '12': 1124,
         '2': 444,
         '3': 279,
         '4': 312,
         '5': 273,
         '6': 279,
         '7': 199,
         '8': 291,
         '9': 1034})

In [6]:
for index, row in df7.iterrows():
    m = month.search(row['Date'])
    for group in m.groups():
        if (group=='1') | (group=='2') | (group=='3'):
            df7.loc[index,'Date'] = 'Jan-Mar'
        elif (group=='4') | (group=='5') | (group=='6'):
            df7.loc[index,'Date'] = 'Apr-Jun'
        elif (group=='7') | (group=='8') | (group=='9'):
            df7.loc[index,'Date'] = 'Jul-Sept'
        else:
            df7.loc[index,'Date'] = 'Oct-Dec'
    
df7.head()

,ID,Text,Illness,Date
0,5.226010e+17,MedSpring's new River North clinic to offer fr...,0,Oct-Dec
1,5.226452e+17,Jk. Flu shots be saving lives but they're anno...,0,Oct-Dec
2,5.226622e+17,"i will get the flu shot anyway, not cause ebol...",1,Oct-Dec
3,5.226739e+17,"Speaking of flu shots, I found out I work with...",0,Oct-Dec
4,5.227409e+17,Who smiles for a flu shot? WWj producer Terri ...,0,Oct-Dec


In [7]:
r_at = re.compile('@[\S]+')

for index, row in df7.iterrows():
    df7.loc[index,'Text'] = r_at.sub('USER_',df7.loc[index,'Text'])
        
df7.head()

,ID,Text,Illness,Date
0,5.226010e+17,MedSpring's new River North clinic to offer fr...,0,Oct-Dec
1,5.226452e+17,Jk. Flu shots be saving lives but they're anno...,0,Oct-Dec
2,5.226622e+17,"i will get the flu shot anyway, not cause ebol...",1,Oct-Dec
3,5.226739e+17,"Speaking of flu shots, I found out I work with...",0,Oct-Dec
4,5.227409e+17,Who smiles for a flu shot? WWj producer Terri ...,0,Oct-Dec


In [8]:
r = re.compile('(http[s]*://[\S^\.]+(?:\.[\S^\.\)]+)+)')
rr = re.compile('&amp;')

for index, row in df7.iterrows():
    df7.loc[index,'Text'] = r.sub('URL_',df7.loc[index,'Text'])
    df7.loc[index,'Text'] = rr.sub('and',df7.loc[index,'Text'])

In [9]:
import re
from collections import Counter

def ngrams(tokens, n):
    output = []
    for i in range(n-1, len(tokens)):
        ngram = ' '.join(tokens[i-n+1:i+1])
        output.append(ngram)
    return output

def skipgrams(tokens, n):
    output = []
    for i in range(n-1, len(tokens)):
        str1 = ''
        for j in range(n-2):
            str1 = str1+' * '
        skipgram = str(tokens[i-n+1])+str1+str(tokens[i])
        output.append(skipgram)
    return output

import itertools

def pairs(words):
    ret = itertools.combinations(words,2)
    return ret
   
import nltk

def parts_of_speech(words):
    tagged = []
    
    try:
        tagged = nltk.pos_tag(words)
        return(tagged)
        
    except Exception as e:
        print(e)

In [10]:
def features(text, ngram_range=(1,3)):
    text = text.lower()      # make the string lowercase
    text = re.sub(r'(.)\1+', r'\1\1', text)     # remove consecutive characters that are repeated more than twice
    
    features_in_text = []   # running list of all features in this instance (can be repeated)
    
    # treat alphanumeric characters as word tokens (removing anything else),
    # and extract all n-grams of length n specified by ngram_range
    
    text_alphanum = re.sub('[^a-z0-9]', ' ', text)
    for n in range(ngram_range[0], ngram_range[1]+1):
        features_in_text += ngrams(text_alphanum.split(), n)
        #remove sentiment
    
    for n in range(3,5):
        features_in_text += skipgrams(text_alphanum.split(), n)
      
    features_in_text += str(pairs(text_alphanum.split()))
    features_in_text += str(parts_of_speech(text_alphanum.split()))
    
    
    text_punc = re.sub('[a-z0-9]', ' ', text)
    features_in_text += ngrams(text_punc.split(), 1)
    
   
    # 'Counter' converts a list into a dictionary whose keys are the list elements 
    #  and the values are the number of times each element appeared in the list
    
    return Counter(features_in_text)

In [11]:
def features_domain(text, df, factor, ngram_range=(1,3)):
    text = text.lower()      # make the string lowercase
    text = re.sub(r'(.)\1+', r'\1\1', text)     # remove consecutive characters that are repeated more than twice
    
    features_in_text = []   # running list of all features in this instance (can be repeated)
    
    # treat alphanumeric characters as word tokens (removing anything else),
    # and extract all n-grams of length n specified by ngram_range
    
    text_alphanum = re.sub('[^a-z0-9]', ' ', text)
    for n in range(ngram_range[0], ngram_range[1]+1):
        features_in_text += ngrams(text_alphanum.split(), n)
        #remove sentiment
    
    for n in range(3,5):
        features_in_text += skipgrams(text_alphanum.split(), n)
      
    features_in_text += str(pairs(text_alphanum.split()))
    features_in_text += str(parts_of_speech(text_alphanum.split()))
    
    
    text_punc = re.sub('[a-z0-9]', ' ', text)
    features_in_text += ngrams(text_punc.split(), 1)
    
    d_features = []
    row = df[df['Text']==text]
    
    season = str(row['Date'])
    
    for feature in features_in_text:
        d_feature = feature + "___" + season
        d_features += d_feature
        
    d_counts = Counter(d_features)
    
    for d_feature in d_counts:
        count = d_counts[d_feature]
        count = float(count) * factor
        d_counts[d_feature] = count
    
    features_in_text_counts = Counter(features_in_text)
    
    return d_counts + features_in_text_counts

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

text_train, text_test, Y_train, Y_test = train_test_split(df7['Text'], df7['Illness'], test_size=0.2, random_state=0)

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction import DictVectorizer

facs = [0.1,0.01,0.001,0.0001,0.00001,0.000001]
fs = []

for f in facs:
    vect = DictVectorizer()
    X_train = vect.fit_transform(features_domain(d, df7, f, ngram_range=(1,3)) for d in text_train)
    base_classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=123)

    # these are the C values we will compare -- don't change this variable

    params = [{'C': [0.01, 0.1, 0.5, 1.0, 2.5, 5.0, 10.0, 50.0, 100.0]}]

    # this performs 5-fold cross-validation with the above classifier and parameter options

    gs_classifier = GridSearchCV(base_classifier, params, cv=5)
    gs_classifier.fit(X_train, Y_train)
    
    print("\n\n\nFactor: %0.6f" % f)

    #print("Best parameter settings:", gs_classifier.best_params_)
    #print("Validation accuracy: %0.6f" % gs_classifier.best_score_)
    
    X_test = vect.transform(features_domain(d, df7, f, ngram_range=(1,3)) for d in text_test)

    print("Validation F1:")
    print(f1_score(Y_train, gs_classifier.predict(X_train), pos_label='1'))
    print("Test F1:")
    print(f1_score(Y_test, gs_classifier.predict(X_test), pos_label='1'))
    fs.append((f1_score(Y_test, gs_classifier.predict(X_test), pos_label='1')))
    
print(fs)
max(fs)




Factor: 0.100000
Validation F1:
0.7783993115318417
Test F1:
0.738819320214669



Factor: 0.010000
Validation F1:
0.8646822883681655
Test F1:
0.7807860262008733



Factor: 0.001000
Validation F1:
0.9216666666666665
Test F1:
0.8034042553191488



Factor: 0.000100
Validation F1:
0.9086710056686963
Test F1:
0.8047538200339558



Factor: 0.000010
Validation F1:
0.8785281204265105
Test F1:
0.7969413763806288



Factor: 0.000001
Validation F1:
0.9216955523073711
Test F1:
0.7989646246764451
[0.738819320214669, 0.7807860262008733, 0.8034042553191488, 0.8047538200339558, 0.7969413763806288, 0.7989646246764451]


0.8047538200339558